# CAFA 5 protein function Prediction with TensorFlow

This notebook walks you through how to train a DNN model using TensorFlow on the CAFA 5 protein function Prediction dataset made available for this competition. 

The objective of the model is to predict the function(aka **GO term ID**) of a set of proteins based on their amino acid sequences and other data.


**Note** : This notebook runs without any GPU. This is because enabling GPUs leaves less RAM memory on the VM and the submission step needs a lot of memory. One point where this would impact is when training the model. With CPU it will take around 2 minutes while on GPU it would take around 30 seconds.

## About the Data

### Protein Sequence

Each protein is composed of dozens or hundreds of amino acids that are linked sequentially. Each amino acid in the sequence may be represented by a one-letter or three-letter code. Thus the sequence of a protein is often notated as a string of letters. 

<img src="https://cityu-bioinformatics.netlify.app/img/tools/protein/pro_seq.png" alt ="Sequence.png" style='width: 800px;' >

Image source - [https://cityu-bioinformatics.netlify.app/](https://cityu-bioinformatics.netlify.app/too2/new_proteo/pro_seq/)

The `train_sequences.fasta` made available for this competitions, contains the sequences for proteins with annotations (labelled proteins).

# Gene Ontology

We can define the functional properties of a proteins using Gene Ontology(GO). Gene Ontology (GO) describes our understanding of the biological domain with respect to three aspects:
1. Molecular Function (MF)
2. Biological Process (BP)
3. Cellular Component (CC)

Read more about Gene Ontology [here](http://geneontology.org/docs/ontology-documentation).

File `train_terms.tsv` contains the list of annotated terms (ground truth) for the proteins in `train_sequences.fasta`. In `train_terms.tsv` the first column indicates the protein's UniProt accession ID (unique protein id), the second is the `GO Term ID`, and the third indicates in which ontology the term appears. 

# Labels of the dataset

The objective of our model is to predict the terms (functions) of a protein sequence. One protein sequence can have many functions and can thus be classified into any number of terms. Each term is uniquely identified by a `GO Term ID`. Thus our model has to predict all the `GO Term ID`s for a protein sequence. This means that the task at hand is a multi-label classification problem. 

# Protein embeddings for train and test data

To train a machine learning model we cannot use the alphabetical protein sequences in`train_sequences.fasta` directly. They have to be converted into a vector format. In this notebook, we will use embeddings of the protein sequences to train the model. You can think of protein embeddings to be similar to word embeddings used to train NLP models.
<!-- Instead, to make calculations and data preparation easier we will use precalculated protein embeddings.
 -->
Protein embeddings are a machine-friendly method of capturing the protein's structural and functional characteristics, mainly through its sequence. One approach is to train a custom ML model to learn the protein embeddings of the protein sequences in the dataset being used in this notebook. Since this dataset represents proteins using amino-acid sequences which is a standard approach, we can use any publicly available pre-trained protein embedding models to generate the embeddings.

There are a variety of protein embedding models. To make data preparation easier, we have used the precalculated protein embeddings created by [Sergei Fironov](https://www.kaggle.com/sergeifironov) using the Rost Lab's T5 protein language model in this notebook. The precalculated protein embeddings can be found [here](https://www.kaggle.com/datasets/sergeifironov/t5embeds). We have added this dataset to the notebook along with the dataset made available for the competition.

To add this to your enviroment, on the right side panel, click on `Add Data` and search for `t5embeds` (make sure that it's the correct [one](https://www.kaggle.com/datasets/sergeifironov/t5embeds)) and then click on the `+` beside it.



# Import the Required Libraries

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import FeatureHasher
import csv
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
import scipy.sparse as sp
import pickle
from Bio import SeqIO
from collections import Counter

# Required for progressbar widget
import progressbar

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
load_model_quim = False
load_matrices = True

# Load the Dataset

In [3]:
train_terms = pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv",sep="\t")
print(train_terms.shape)
print(train_terms.head())

# Count unique terms by aspect
term_counts = train_terms.groupby('aspect')['term'].nunique()
print(term_counts)

# Group terms by aspect and count the occurrences
term_aspect_counts = train_terms.groupby('term')['aspect'].nunique()
print('number of unique GO terms:',len(train_terms['term'].unique()))

# Filter terms that appear in only one aspect
multiple_aspect_terms = term_aspect_counts[term_aspect_counts > 1]

if multiple_aspect_terms.empty:
    print('There are no terms that appear in only one aspect')


(5363863, 3)
      EntryID        term aspect
0  A0A009IHW8  GO:0008152    BPO
1  A0A009IHW8  GO:0034655    BPO
2  A0A009IHW8  GO:0072523    BPO
3  A0A009IHW8  GO:0044270    BPO
4  A0A009IHW8  GO:0006753    BPO
aspect
BPO    21285
CCO     2957
MFO     7224
Name: term, dtype: int64
number of unique GO terms: 31466
There are no terms that appear in only one aspect


In [4]:
# Step 1: Assign unique identifiers to terms
unique_terms = train_terms['term'].unique()
term_to_id = {term: idx for idx, term in enumerate(unique_terms)}

if not load_matrices:
    
    # Step 2: Create an empty dictionary to store encoded sparse matrices
    encoded_matrices = {}

    # Step 3: Iterate over EntryID groups and encode terms
    grouped_entries = train_terms.groupby('EntryID')
    for entry_id, group in grouped_entries:
        # Initialize an empty sparse matrix for the EntryID group
        num_terms = len(group)
        encoded_matrix = sp.lil_matrix((1, len(unique_terms)), dtype=np.int8)

        # Encode terms in the sparse matrix
        for _, row in group.iterrows():
            term = row['term']
            term_id = term_to_id[term]
            encoded_matrix[0, term_id] = max(encoded_matrix[0, term_id], 1)  # Using max encoding

        # Store the encoded sparse matrix for the EntryID
        encoded_matrices[entry_id] = encoded_matrix.tocsr()

    # Save the encoded_matrices dictionary
    with open('encoded_matrices.pkl', 'wb') as file:
        pickle.dump(encoded_matrices, file)

    # Example usage: Retrieve encoded matrix for a specific EntryID
    entry_id = 'A0A009IHW8'
    encoded_matrix = encoded_matrices[entry_id]

    # Example usage: Decode a specific term for the EntryID
    term_id = 42  # Example term ID
    term = unique_terms[term_id]
    decoded_term = term

    # Print the encoded matrix for the EntryID
    print(encoded_matrix)

    # Print the decoded term for the EntryID
    print(decoded_term)

else:
    
    # Load the encoded_matrices dictionary from file
    with open('/kaggle/input/modelcafa5/encoded_matrices.pkl', 'rb') as file:
        encoded_matrices = pickle.load(file)

In [5]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from Bio import SeqIO

def get_fasta_1(fasta_file):
    # Create empty lists to store the sequences and protein IDs
    sequences = []
    protein_ids = []
    max_length = 60

    # Open and read the .fasta file
    with open(fasta_file, "r") as handle:
        # Use the SeqIO.parse() function to parse the .fasta file
        # SeqIO.parse() returns an iterator over the sequences in the file
        records = SeqIO.parse(handle, "fasta")

        # Iterate over the sequences
        for record in records:
            # Access the RegisterID and sequence
            register_id = record.id
            sequence = str(record.seq)  # Convert sequence to string if needed

            # Add the sequence and protein ID to the lists
            sequences.append(sequence[:max_length])  # Only take the first 20 characters
            protein_ids.append(register_id)

    # Create a set to store unique amino acids, excluding padding character '-'
    amino_acids = set()
    
    # Iterate over the sequences to determine unique amino acids
    for sequence in sequences:
        amino_acids.update(sequence)

    # Determine the maximum sequence length and the number of unique amino acids
    num_amino_acids = len(amino_acids)

    # Create a list to store the flattened sparse matrices
    sparse_matrices = []
    
    # Iterate over the sequences
    for sequence in sequences:
        # Truncate or pad the sequence to a length of 20 amino acids
        truncated_sequence = sequence.ljust(max_length, '-')

        # Encode the sequence as a sparse matrix
        row_indices = []
        col_indices = []
        data = []
        for i, amino_acid in enumerate(truncated_sequence):
            if amino_acid != '-':
                amino_acid_index = list(amino_acids).index(amino_acid)
                row_indices.append(i)
                col_indices.append(amino_acid_index)
                data.append(1)

        # Create the sparse matrix
        dataset = csr_matrix((data, (row_indices, col_indices)), shape=(max_length, num_amino_acids))
        
        # Append the sparse matrix to the list
        sparse_matrices.append(dataset.toarray().flatten())

    # Create a pandas DataFrame from the list of flattened sparse matrices
    column_names = [f"Position_{i+1}_{aa}" for i in range(max_length) for aa in amino_acids]
    df = pd.DataFrame(sparse_matrices, columns=column_names)

    # Add the protein IDs as a column in the DataFrame
    df['EntryID'] = protein_ids

    # Return the pandas DataFrame
    return df


In [6]:
def get_fasta_2(data1,fasta_file):
    # Create an empty list to store the encoded data
    encoded_data = []

    # Open and read the .fasta file
    with open(fasta_file, "r") as handle:
        # Use the SeqIO.parse() function to parse the .fasta file
        # SeqIO.parse() returns an iterator over the sequences in the file
        records = SeqIO.parse(handle, "fasta")

        # Iterate over the sequences
        for record in records:
            # Access the RegisterID and sequence
            register_id = record.id
            sequence = str(record.seq)  # Convert sequence to string if needed

            # Create a Counter object for the sequence
            sequence_counts = Counter(sequence)

            # Duplicate the sequence_counts columns
            duplicated_counts = {k + "_copy": v for k, v in sequence_counts.items()}

            # Append the RegisterID, sequence_counts, and duplicated_counts to the encoded data
            encoded_data.append({"EntryID": register_id, **sequence_counts, **duplicated_counts})

    # Create a pandas DataFrame from the encoded data
    data = pd.DataFrame(encoded_data)

    # Print the resulting DataFrame
    data = data.fillna(0)
    
    # Merge the two DataFrames on the 'EntryID' column
    merged_df = pd.merge(data1, data, on='EntryID')
    
    return merged_df

In [7]:
# Path to the .fasta file
fasta_file = "/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta"

data1 = get_fasta_1(fasta_file)
data = get_fasta_2(data1,fasta_file)
    
data= data.fillna(0)
data.to_csv('data.csv', index=False)
data

,Position_1_T,Position_1_R,Position_1_Q,Position_1_G,Position_1_M,Position_1_K,Position_1_P,Position_1_L,Position_1_I,Position_1_V,...,U,U_copy,O,O_copy,X,X_copy,B,B_copy,Z,Z_copy
0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142241,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142242,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142243,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142244,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
print(len(data['EntryID']),len(data['EntryID'].unique()))

142246 142246


In [9]:
import tensorflow as tf
from tensorflow.keras import layers

if not load_model_quim:
    
    # Define the model architecture
    model = tf.keras.Sequential()
    model.add(layers.Reshape((-1, 1), input_shape=(len(data.columns)-1,)))  # Reshape input to (num_features, 1)
    model.add(layers.Conv1D(64, kernel_size=3, activation='relu'))  # Convolutional layer
    model.add(layers.Conv1D(32, kernel_size=3, activation='relu'))  # Convolutional layer
    model.add(layers.Flatten())  # Flatten the output
    model.add(layers.Dense(128, activation='relu'))  # Hidden layer
    model.add(layers.Dense(64, activation='relu'))  # Hidden layer
    model.add(layers.Dense(31466, activation='sigmoid'))  # Output layer

    # Set the learning rate
    learning_rate = 0.0005

    # Create a custom optimizer with the desired learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Compile the model with the custom optimizer
    model.compile(optimizer=optimizer, loss='binary_crossentropy')

    i = 0
    # Iterate over each EntryID and update the model
    for entry_id in data['EntryID'].unique():
        # Get the encoded matrix for the current EntryID
        encoded_matrix = encoded_matrices[entry_id]

        # Prepare the input features (data) for training
        input_features = data.loc[data['EntryID'] == entry_id].drop(columns='EntryID').values

        # Update the model with new data for the current EntryID
        model.train_on_batch(input_features, encoded_matrix.toarray())

        print(i, entry_id, end="\r")
        i += 1

    # Save the model
    model.save('model_new_2.h5')
    print("Model saved after", i, "iterations")
    
else:

    # Load the model
    model = load_model('/kaggle/input/modelcafa5/model_new_2.h5')
    
# Example usage: Make predictions for a specific EntryID
entry_id = 'A0A009IHW8'
input_features = data.loc[data['EntryID'] == entry_id].drop(columns='EntryID').values
prediction = model.predict(input_features)

# Example usage: Decode the predicted matrix for the EntryID
decoded_prediction = prediction.argmax(axis=1)

# Decode the predicted terms using the term_to_id dictionary
decoded_terms = [unique_terms[term_id] for term_id in decoded_prediction]

# Print the decoded terms
print(decoded_terms)

Model saved after 142246 iterations
1/1 [==============================] - 0s 412ms/step
['GO:0008150']


In [10]:
prediction

array([[1.7268150e-01, 1.2313638e-02, 1.7045165e-04, ..., 4.3388412e-05,
        4.3407203e-05, 4.3677519e-05]], dtype=float32)

In [11]:
threshold = 0.01  # Probability threshold

# Apply thresholding to the prediction probabilities
thresholded_prediction = prediction.copy()
thresholded_prediction[thresholded_prediction < threshold] = 0

# Get the indices of the terms with probabilities above the threshold
term_indices = thresholded_prediction.argsort(axis=1)[:, ::-1]

# Retrieve the terms corresponding to the indices
predicted_terms = [[unique_terms[idx] for idx in indices if thresholded_prediction[i, idx] > 0] for i, indices in enumerate(term_indices)]

# Print the predicted terms for each sample
for terms in predicted_terms:
    print(terms)


['GO:0008150', 'GO:0005575', 'GO:0110165', 'GO:0005622', 'GO:0003674', 'GO:0043226', 'GO:0009987', 'GO:0043229', 'GO:0043227', 'GO:0043231', 'GO:0005488', 'GO:0005737', 'GO:0005515', 'GO:0005634', 'GO:0065007', 'GO:0050789', 'GO:0032502', 'GO:0048856', 'GO:0032501', 'GO:0050794', 'GO:0008152', 'GO:0050896', 'GO:0071704', 'GO:0044237', 'GO:0044238', 'GO:0043170', 'GO:0006807', 'GO:0071840', 'GO:0007275', 'GO:0016043', 'GO:0019222', 'GO:0006950', 'GO:0043228', 'GO:0043232', 'GO:0060255', 'GO:0003824', 'GO:0031981', 'GO:0031974', 'GO:0070013', 'GO:0043233', 'GO:0006996', 'GO:0010468', 'GO:0032991', 'GO:0051716', 'GO:1901363', 'GO:0048869', 'GO:0097159', 'GO:0031323', 'GO:0000003', 'GO:0080090', 'GO:0030154', 'GO:0006725', 'GO:0048731', 'GO:1901360', 'GO:0005829', 'GO:0051171', 'GO:0034641', 'GO:0003676', 'GO:0046483', 'GO:0048513', 'GO:0005654', 'GO:0071944', 'GO:0016020', 'GO:0048468', 'GO:0006139', 'GO:0048519', 'GO:0090304', 'GO:0048518', 'GO:0033554', 'GO:0022414', 'GO:0019219', 'GO:0

In [12]:
set(train_terms[train_terms['EntryID'] == 'A0A009IHW8']['term']) - set(terms)

{'GO:0003953',
 'GO:0006163',
 'GO:0006195',
 'GO:0006753',
 'GO:0009117',
 'GO:0009166',
 'GO:0016798',
 'GO:0016799',
 'GO:0019362',
 'GO:0019364',
 'GO:0019637',
 'GO:0019674',
 'GO:0019677',
 'GO:0046434',
 'GO:0046496',
 'GO:0055086',
 'GO:0072521',
 'GO:0072523',
 'GO:0072524',
 'GO:0072526',
 'GO:1901292',
 'GO:1901565'}

In [13]:
test_file = '/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta'
testA = get_fasta_1(test_file)
test1 = get_fasta_2(testA,test_file)

In [14]:
# Check if any columns in data are missing in test
missing_columns = list(set(data.columns) - set(test1.columns))

# Create a DataFrame with missing columns and fill with 0
missing_df = pd.DataFrame(0, index=test1.index, columns=missing_columns)

# Concatenate missing_df with test1
test1 = pd.concat([test1, missing_df], axis=1)

# Remove additional columns from test1
extra_columns = list(set(test1.columns) - set(data.columns))
test1 = test1.drop(columns=extra_columns)

# Reorder columns in test1 to match the order in data
test = test1[data.columns]

# Print the updated DataFrame
test

,Position_1_T,Position_1_R,Position_1_Q,Position_1_G,Position_1_M,Position_1_K,Position_1_P,Position_1_L,Position_1_I,Position_1_V,...,U,U_copy,O,O_copy,X,X_copy,B,B_copy,Z,Z_copy
0,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141862,0,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
141863,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
141864,0,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
141865,0,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Define the protein IDs
protein_ids = test['EntryID']
print("length protein ids:", len(protein_ids))

# Create an empty list to store data for submission
submission_data = []

batch_size = 1000  # Adjust the batch size as per your memory constraints

total_terms = 0
proteins = 0

# Process the dataset in batches
for start_index in range(0, len(protein_ids), batch_size):
    end_index = start_index + batch_size
    
    # Get the protein IDs and input features for the current batch
    batch_protein_ids = protein_ids[start_index:end_index]
    batch_input_features = test.loc[test['EntryID'].isin(batch_protein_ids)].drop(columns='EntryID').values
    
    # Make predictions for the current batch using the trained model
    batch_predictions = model.predict(batch_input_features, verbose=0)
    
    # Iterate over the proteins in the batch
    for i, protein_id in enumerate(batch_protein_ids):
        # Get the predictions and input features for the current protein
        protein_predictions = batch_predictions[i]
        protein_input_features = batch_input_features[i]
        
        # Apply thresholding to the predictions
        thresholded_predictions = protein_predictions > threshold
        
        # Retrieve the indices of the terms with predictions above the threshold
        term_indices = thresholded_predictions.nonzero()[0]
        
        # Retrieve the terms corresponding to the indices
        predicted_terms = [unique_terms[idx] for idx in term_indices]
        
        total_terms += len(term_indices)
        proteins += 1
        print('total terms:', total_terms, 'total proteins:', proteins, end='\r')
        
        # Generate the GO term IDs based on the protein ID
        go_term_ids = [f'{term}' for term in predicted_terms]
        
        # Append the protein ID, GO term IDs, and predictions to the submission data
        submission_data.extend(zip([protein_id] * len(go_term_ids), go_term_ids, protein_predictions[term_indices]))
        
    # Clear variables to release memory
    del batch_protein_ids, batch_input_features, batch_predictions

# Create the submission DataFrame
df_submission = pd.DataFrame(submission_data, columns=['Protein Id', 'GO Term Id', 'Prediction'])

# Save the submission DataFrame to a TSV file
df_submission.to_csv("submission.tsv", header=False, index=False, sep="\t")
df_submission

length protein ids: 141867


,Protein Id,GO Term Id,Prediction
0,Q9CQV8,GO:0008152,0.118578
1,Q9CQV8,GO:0044237,0.103990
2,Q9CQV8,GO:1901360,0.063164
3,Q9CQV8,GO:0008150,0.601585
4,Q9CQV8,GO:1901564,0.045255
...,...,...,...
53007588,A0A3G2FQK2,GO:0048018,0.018682
53007589,A0A3G2FQK2,GO:0030545,0.018369
53007590,A0A3G2FQK2,GO:0030546,0.018605
53007591,A0A3G2FQK2,GO:0003729,0.016038
